In [1]:
from collections import Counter
import nltk
from nltk.tokenize import TweetTokenizer
import pandas as pd
import re as regex
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from time import time
import csv
import preprocessor as p

d:\python 3.6.4\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [21]:
dataset = pd.read_csv('source_replies_dataframe.csv', delimiter=",",quoting=csv.QUOTE_ALL, encoding="utf8")

In [22]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

In [161]:
snttrain = dataset[:4000]
snttest = dataset[4000:]

In [162]:
sentimenttrain = []
for index, row in snttrain.iterrows():
    training = row['reply_text']
    snt = analyser.polarity_scores(training)
    sentimenttrain.append(snt)

In [166]:
sentimenttest = []
for index, row in snttest.iterrows():
    testing = row['reply_text']
    snt = analyser.polarity_scores(testing)
    sentimenttest.append(snt)

In [165]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
trainsnt = vec.fit_transform(sentimenttrain).toarray()

In [167]:
testsnt = vec.fit_transform(sentimenttest).toarray()

In [168]:
#testsnt.shape                                                   #sentiment array: trainsnt, testsnt

(221, 4)

In [24]:
keep_cols=['topic','source_text','reply_text','nested','reply_type']
newdata = dataset[keep_cols]

In [25]:
temp = [] 
for x in newdata['reply_text']:
    temp.append(p.clean(x))
temp = pd.Series(temp)
newdata['reply_text'] = temp

d:\python 3.6.4\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
import json
def cuewordscount(countfile):
    fw = open("count.txt", "w")
    with open("cue_word_list.txt", "r") as f1:
        cue_words = []
        for line in f1:
            cue_words.append(line.strip())
        wordcount = dict((x, 0) for x in cue_words)
    
        for index, row in countfile.iterrows():
            tw = row["reply_text"]
            for w in re.findall(r"\w+", tw):
                if w in wordcount:
                    wordcount[w] += 1
            fw.write ("tweet" + str(index) + ":")
            fw.write (json.dumps(wordcount))
            fw.write ("\n")
            wordcount = dict((x, 0) for x in cue_words)
    fw.close()

In [27]:
keep_cols=['reply_text','reply_type']
tokenizedtw = newdata[keep_cols]

In [15]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

In [16]:
temp = [] 
for x in tokenizedtw['reply_text']:
    temp.append(tknzr.tokenize(x))
temp = pd.Series(temp)
tokenizedtw['reply_text'] = temp

g:\python 3.6.2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [200]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [204]:
keep_cols=['reply_text','reply_type']
tokenizedtw = newdata[keep_cols]

In [205]:
tokenizedtw

,reply_text,reply_type
0,Identical but irrelevant RT : Did anyone think...,comment
1,"no, because that would require reasoning and t...",support
2,,deny
3,"is it me, or the dude in the security cam phot...",comment
4,it's not him lmao the store owners already sai...,comment
5,my God sucks,comment
6,. yes. They don't match. Flip flops + hat in s...,support
7,It's a good thing u went to detective school.....,comment
8,yes hat and sandals vs no hat and tennis shoes,support
9,I thought the same thing.,support


In [30]:
tokenizedtw.loc[tokenizedtw['reply_type']=='support','reply_type']= 0

d:\python 3.6.4\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
d:\python 3.6.4\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
tokenizedtw.loc[tokenizedtw['reply_type']=='comment','reply_type']= 1

d:\python 3.6.4\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
d:\python 3.6.4\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
tokenizedtw.loc[tokenizedtw['reply_type']=='deny','reply_type']= 2

d:\python 3.6.4\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
d:\python 3.6.4\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [33]:
tokenizedtw.loc[tokenizedtw['reply_type']=='query','reply_type']= 3

d:\python 3.6.4\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
d:\python 3.6.4\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
tokenizedtw_x = tokenizedtw['reply_text']
tokenizedtw_y = tokenizedtw['reply_type']

In [77]:
x_train = tokenizedtw_x[:4000]
x_test = tokenizedtw_x[4000:]
y_train = tokenizedtw_y[:4000]
y_test = tokenizedtw_y[4000:]

In [79]:
cv = CountVectorizer()

In [80]:
tv = TfidfVectorizer(stop_words='english') #remove stopwords

In [89]:
x_traintv = tv.fit_transform(x_train)
x_testtv = tv.transform(x_test)

In [90]:
y_train = y_train.astype('int')

In [132]:
from sklearn import ensemble
clf = ensemble.GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1,max_depth=1, random_state=0)

In [133]:
clf.fit(x_traintv, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=1,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=1000,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False)

In [134]:
GBpredcv = clf.predict(x_testtv)

In [135]:
gold = np.array(y_test)

In [118]:
def acc(pred):
    count = 0
    for i in range (len(pred)):
        if pred[i]==gold[i]:
            count+=1
    accu = count/len(pred)
    return accu

In [136]:
acc(GBpredcv)

0.665158371040724

In [178]:
x_traintvsnt = x_traintv.toarray()
x_traintvsnt = np.concatenate((x_traintvsnt, trainsnt), axis=1)

In [179]:
x_testtvsnt = x_testtv.toarray()
x_testtvsnt = np.concatenate((x_testtvsnt, testsnt), axis=1)

In [180]:
x_testtvsnt.shape

(221, 5552)

In [185]:
from sklearn import ensemble
clf = ensemble.GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1,max_depth=1, random_state=0)

In [186]:
clf.fit(x_traintvsnt, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=1.0, loss='deviance', max_depth=1,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=1000,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False)

In [187]:
GBpredcvsnt = clf.predict(x_testtvsnt)

In [188]:
acc(GBpredcvsnt)

0.5520361990950227

In [39]:
#sentiment array: trainsnt, testsnt

0    Identical but irrelevant RT : Did anyone think...
1    no, because that would require reasoning and t...
2                                                     
3    is it me, or the dude in the security cam phot...
4    it's not him lmao the store owners already sai...
Name: reply_text, dtype: object